In [ ]:
import config
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from IPython.display import display
import json
import datetime
import math

from utils.data_exploration_utils import kruskal_wallis_analysis, barplots
from utils.hdbscan_utils import plot_hdbscan, plot_hdbscan_highlight_kl, make_cluster_color_map
from utils.plot_utils import plotly_hdbscan_highlight_kl

In [ ]:
STAGE = 'ss'
MOD_PREFIX = "mod_smallimg3"
NEPOCH = 'latest'


DATAPATH = config.OUTPUT_PATH
base_dir = config.RAW_DATA_PATH
img_path = config.SCHULTHESS_DATAPATH
proc_dir = config.PROC_DATA_PATH

feature = 'rawq'
methods = 'comb_modalities'
folder = "2026-01-17_hdbscan"
run = "run28"  

# methods = 'pipeline'
# folder = "2025-11-19_hdbscan"
# run = "run10"  

anomalyscore_metric = "centre_mean"
cluster_col = "cluster_label"

## Load HDBSCAN Data

In [ ]:
today = datetime.date.today()
folder_date = folder.split('_')[0]

filepath = os.path.join(proc_dir, folder, methods, run)
save_path = os.path.join(filepath, "img")
os.makedirs(save_path, exist_ok=True)

hdbscan_df = pd.read_csv(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_severity_scores.csv'))


with open(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_scaled_model_info.json')) as f:
    model_info= json.load(f)

In [ ]:
if methods =='comb_modalities':
    train = hdbscan_df[hdbscan_df['train_test']=='train']
    test = hdbscan_df[hdbscan_df['train_test']=='test']

    ids = train['id']
    ids_test = test['id']
else:
    ids = hdbscan_df['id']



In [ ]:
hdbscan_df['cluster_label'].value_counts()

In [ ]:
hdbscan_df.columns

## Load Embeddings

In [ ]:
# embeddings_path = os.path.join(filepath, "X_umap_embeddings.npy")
# X_umap = np.load(embeddings_path)

# ids = model_info['files']['ids']
# id_to_index = {id_: index for index, id_ in enumerate(ids)}

## Load MRI Data

In [ ]:
mri = pd.read_csv(os.path.join(base_dir, '2025-09-25_mrismall.csv'))

## Create Combined Data

In [ ]:
hdbscan_df = hdbscan_df.merge(mri, on = 'id', how='left')

In [ ]:
hdbscan_df.to_csv(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_scaled_allpoints_wKL_mri_newclusters.csv'), index=False)

# Anomaly Score distribution

In [ ]:
# comb2 = combined.iloc[:, :-3]

# Boxplot
plt.figure(figsize=(6, 4))
plt.hist(hdbscan_df['mean'], bins=20)
plt.title('Distribution of Mean Values')
plt.xlabel('Mean')
plt.ylabel('Frequency')
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(x='cluster_label', y='mean', data=hdbscan_df, order=sorted(hdbscan_df['cluster_label'].unique()))
plt.title('Boxplot of Mean Anomaly Scores by Cluster Label')
plt.xlabel('Cluster Label') 
plt.ylabel('Mean Anomaly Score')
plt.show()

In [ ]:
print(f"Min. Anomaly Score: {hdbscan_df['mean'].min():.3f}")
print(f"Max. Anomaly Score: {hdbscan_df['mean'].max():.3f}")

In [ ]:
for cluster in hdbscan_df['cluster_label'].unique():
    cluster_data = hdbscan_df[hdbscan_df['cluster_label'] == cluster]['mean']
    print(f"Cluster {cluster}: n={len(cluster_data)}, mean={cluster_data.mean():.3f}, std={cluster_data.std():.3f}, min={cluster_data.min():.3f}, max={cluster_data.max():.3f}")

In [ ]:
mri_columns = [ 'mri_operator',
 'mri_side',
 'mri_bml_yn',
 'mri_cart_yn',
 'mri_osteo_yn',
 'mri_syn_yn',
 'mri_mnsc_yn',
 'mri_lig_yn']


# Some Exploration

In [ ]:
hdbscan_df['cluster_label'].value_counts().reset_index().sort_values('cluster_label')

In [ ]:
df=hdbscan_df

In [ ]:
values = df['cluster_label'].value_counts().reset_index().sort_values(by='cluster_label')

plt.bar(values['cluster_label'], values['count'], color = 'skyblue')
plt.xlabel('Cluster Label')
plt.ylabel('Count')
plt.show()

## Feature Analysis Clusters

In [ ]:
import math

In [ ]:
cols = ['pain', 'age', 'ce_bmi', 'ce_fm']
col_names = ['Pain', 'Age', 'BMI', 'Body Fat Percentage']

n = 2
rows = math.ceil(n / 2)

plt.figure(figsize=(5*n, 5))

for i, col in enumerate(cols, 1):
    rows = math.ceil(len(cols) / n)
    plt.subplot(rows, n, i)
    sns.violinplot(
        data=df,
        x='old_cluster',
        y=col,
        palette='Set3',
        inner='quartile'
    )
    plt.title(f'{col_names[i-1]} by Cluster Label')
    plt.xlabel('cluster_label')
    plt.ylabel(col_names[i-1])
    sns.despine()

plt.tight_layout()
plt.show()

In [ ]:
cols = ['OKS_score', 
       'KOOS_pain', 'KOOS_symptoms', 'KOOS_sport', 'KOOS_adl',
       'KOOS_qol']
n = 2
rows = math.ceil(n / 2)

plt.figure(figsize=(5*n, 10))

for i, col in enumerate(cols, 1):
    rows = math.ceil(len(cols) / n)
    plt.subplot(rows, n, i)
    sns.violinplot(
        data=df,
        x='old_cluster',
        y=col,
        palette='Set3',
        inner='quartile'
    )
    plt.title(f'{col} by Cluster Label')
    plt.xlabel('cluster_label')
    plt.ylabel(col)
    sns.despine()

plt.tight_layout()
plt.show()

In [ ]:
cols = ['OKS_score', 
       'KOOS_pain', 'KOOS_symptoms', 'KOOS_sport', 'KOOS_adl',
       'KOOS_qol']
n = 2
rows = math.ceil(n / 2)

plt.figure(figsize=(5*n, 10))

for i, col in enumerate(cols, 1):
    rows = math.ceil(len(cols) / n)
    plt.subplot(rows, n, i)
    sns.boxplot(
        data=df,
        x='old_cluster',
        y=col,
        palette='Set3',
        # inner='quartile'
    )
    plt.title(f'{col} by Cluster Label')
    plt.xlabel('cluster_label')
    plt.ylabel(col)
    sns.despine()

plt.tight_layout()
plt.show()

In [ ]:
df_wonoise = df[df['old_cluster'] != -1]
cols = [
    'OKS_score',
    'KOOS_pain', 'KOOS_symptoms', 'KOOS_sport',
    'KOOS_adl', 'KOOS_qol'
]

n_cols = 2
n_rows = math.ceil(len(cols) / n_cols)

plt.figure(figsize=(6 * n_cols, 4 * n_rows))

for i, col in enumerate(cols, 1):
    plt.subplot(n_rows, n_cols, i)

    # jitter KL-score slightly to avoid overlap
    jitter = np.random.normal(0, 0.08, size=len(df_wonoise))

    sns.scatterplot(
        data=df_wonoise,
        x=df_wonoise['KL-Score'] + jitter,
        y=col,
        hue='old_cluster',
        palette='Set2',
        alpha=0.5,
        s=60,
        edgecolor='none'
    )

    plt.title(f'{col} vs KL-score')
    plt.xlabel('KL score')
    plt.ylabel(col)
    sns.despine()

    # clean legend (only once)
    if i != 1:
        plt.legend([], [], frameon=False)

plt.tight_layout()
plt.show()


In [ ]:
cols = [
    'OKS_score',
    'KOOS_pain', 'KOOS_symptoms', 'KOOS_sport',
    'KOOS_adl', 'KOOS_qol'
]

# make cluster labels explicit + ordered
cluster_order = sorted(df_wonoise["old_cluster"].unique())

# define a fixed palette (important for legend correctness)
cluster_colors ={
    0: '#bebada',
    1: '#80b1d3',
    2: '#fccde5',
    3: '#bc80bd',
    4: '#ffed6f'}


n_cols = 2
n_rows = math.ceil(len(cols) / n_cols)

fig, axes = plt.subplots(
    n_rows,
    n_cols,
    figsize=(6 * n_cols, 4.5 * n_rows),
    squeeze=False
)

axes = axes.flatten()

for i, col in enumerate(cols):
    ax = axes[i]

    sns.barplot(
        data=df_wonoise,
        x="KL-Score",
        y=col,
        hue="old_cluster",
        hue_order=cluster_order,
        estimator=np.median,
        errorbar=("pi", 50),   # IQR
        # palette=cluster_colors,
        palette = 'Set2',
        capsize=0.15,
        ax=ax
    )

    ax.set_title(col, fontsize=13)
    ax.set_xlabel("KL score")
    ax.set_ylabel(col)
    sns.despine(ax=ax)

    # remove per-axis legend
    ax.legend_.remove()

# -----------------------------
# Global legend (from palette)
# -----------------------------
palette = sns.color_palette("Set2", n_colors=len(cluster_order))

handles = [
    plt.Line2D([0], [0], marker='s', linestyle='',
               color=palette[c], markersize=12)
    for c in cluster_order
]

labels = [f"Cluster {c}" for c in cluster_order]

fig.legend(
    handles,
    labels,
    title="Cluster",
    loc="upper center",
    bbox_to_anchor=(0.5, 1.02),
    ncol=len(cluster_order),
    frameon=False
)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


In [ ]:
cols = ['mri_cart_yn', 'mri_osteo_yn', 'mri_bml_yn']
n_cols = 2
n_rows = math.ceil(len(cols) / n_cols)

fig, axes = plt.subplots(
    n_rows,
    n_cols,
    figsize=(6 * n_cols, 4.5 * n_rows),
    squeeze=False
)

axes = axes.flatten()

for i, col in enumerate(cols):
    ax = axes[i]

    sns.barplot(
        data=df_wonoise,
        x="KL-Score",
        y=col,
        hue="old_cluster",
        hue_order=cluster_order,
        estimator=np.median,
        errorbar=("pi", 50),   # IQR
        palette=cluster_colors,
        capsize=0.15,
        ax=ax
    )

    ax.set_title(col, fontsize=13)
    ax.set_xlabel("KL score")
    ax.set_ylabel(col)
    sns.despine(ax=ax)

    # remove per-axis legend
    ax.legend_.remove()

# -----------------------------
# Global legend (from palette)
# -----------------------------
handles = [
    plt.Line2D([0], [0], marker='s', linestyle='',
               color=cluster_colors[c], markersize=12)
    for c in cluster_order
]

labels = [f"Cluster {c}" for c in cluster_order]

fig.legend(
    handles,
    labels,
    title="Cluster",
    loc="upper center",
    bbox_to_anchor=(0.5, 1.02),
    ncol=len(cluster_order),
    frameon=False
)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### Cluster Label vs KL-Score

In [ ]:
# scatterplot
color_map = make_cluster_color_map(df['KL-Score'].unique())
sns.set_theme(style="whitegrid", font_scale=1.2)
plt.figure(figsize=(20, 10))
sns.catplot(data = df[df['cluster_label']!=-1], x='cluster_label', y='probability', hue='KL-Score', palette='Set3', jitter = 0.3)
plt.show()

In [ ]:
sns.set_theme(style="whitegrid", font_scale=1.2)

df2 = df[df['cluster_label'] != -1]

g = sns.catplot(
    data=df,
    x='cluster_label',
    y='probability',
    hue='cluster_label',          # <- adds legend
    kind='box',
    col='cluster_label',
    col_wrap=2,
    palette='Set3',
    height=5,
    aspect=1.2,
)

g.set_titles("Cluster {col_name}")
g.set_axis_labels("cluster_label", "Probability")
g.tight_layout()
plt.show()



In [ ]:
kls = sorted(df['cluster_label'].unique())
kls = [kl for kl in kls if kl != -1]  # exclude noise

ncols = 2
nrows = math.ceil(len(kls)/ncols)
fig, ax = plt.subplots(nrows, ncols, figsize=(20, 4*nrows), sharey=True)
ax = np.ravel(ax)  # flatten to 1D

for idx, kl in enumerate(kls):
    sns.boxplot(
        data=df[df['cluster_label'] == kl],
        x='KL-Score', y='probability',
        ax=ax[idx], color=color_map[kl]
    )
    ax[idx].set_title(f"cluster_label = {kl}")

# hide any unused axes
for j in range(len(kls), len(ax)):
    ax[j].set_visible(False)

plt.tight_layout()
plt.savefig(os.path.join(save_path, f'{folder}_{run}_probability_cluster_klscore_v2_rawq.png'))
plt.show()

In [ ]:
sns.set_theme(style="whitegrid", font_scale=1.2)
plt.figure(figsize=(12, 7))

ax = sns.countplot(
    data=df,
    x='cluster_label',
    hue='KL-Score',
    palette='Set3'
)

# Add counts above bars
for container in ax.containers:
    ax.bar_label(container, fmt='%d', padding=2)

plt.xlabel("Cluster Label")
plt.ylabel("Count")
plt.title("Distribution of KL-Score Across Clusters")

plt.show()

In [ ]:
sns.set_theme(style="whitegrid", font_scale=1.2)
plt.figure(figsize=(12, 7))

ax = sns.countplot(
    data=df,
    x='old_cluster',
    hue='KL-Score',
    palette='Set3'
)

# Add counts above bars
for container in ax.containers:
    ax.bar_label(container, fmt='%d', padding=2)

plt.xlabel("Cluster Label")
plt.ylabel("Count")
plt.title("Distribution of KL-Score Across Clusters")

plt.show()

In [ ]:
sns.set_theme(style="whitegrid", font_scale=1.2)
plt.figure(figsize=(12, 7))

ax = sns.countplot(
    data=train,
    x='cluster_label',
    hue='KL-Score',
    palette='Set3'
)

# Add counts above bars
for container in ax.containers:
    ax.bar_label(container, fmt='%d', padding=2)

plt.xlabel("Cluster Label")
plt.ylabel("Count")
plt.title("Distribution of KL-Score Across Clusters - Train")

plt.show()

In [ ]:
sns.set_theme(style="whitegrid", font_scale=1.2)
plt.figure(figsize=(12, 7))

ax = sns.countplot(
    data=test,
    x='cluster_label',
    hue='KL-Score',
    palette='Set3'
)

# Add counts above bars
for container in ax.containers:
    ax.bar_label(container, fmt='%d', padding=2)

plt.xlabel("Cluster Label")
plt.ylabel("Count")
plt.title("Distribution of KL-Score Across Clusters - Test")

plt.show()

## Cluster Label vs MRI Data

In [ ]:
summary = df.groupby('cluster_label')[['mri_cart_yn', 'mri_osteo_yn', 'mri_bml_yn']].mean()

In [ ]:
summary

In [ ]:
summary_long = summary.reset_index().melt(
    id_vars="cluster_label",
    value_vars=["mri_cart_yn", "mri_osteo_yn", "mri_bml_yn"],
    var_name="MRI_feature",
    value_name="proportion_positive"
)
sns.set_theme(style="whitegrid", font_scale=1.2)

plt.figure(figsize=(15, 6))

sns.barplot(
    data=summary_long,
    x="cluster_label",
    y="proportion_positive",
    hue="MRI_feature",
    palette="Set3"
    , width=0.6
)

plt.xlabel("Cluster Label", fontsize=14)
plt.ylabel("Proportion Positive", fontsize=14)
plt.title("MRI Pathology Prevalence per Cluster", fontsize=16)
plt.legend(
    title="MRI Finding",
    bbox_to_anchor=(1.05, 1),
    loc="upper left"
)
plt.tight_layout()
plt.show()

### KL-Score MRI Data

In [ ]:
summary = df.groupby('KL-Score')[['mri_cart_yn', 'mri_osteo_yn', 'mri_bml_yn']].mean()
summary_long = summary.reset_index().melt(
    id_vars="KL-Score",
    value_vars=["mri_cart_yn", "mri_osteo_yn", "mri_bml_yn"],
    var_name="MRI_feature",
    value_name="proportion_positive"
)
sns.set_theme(style="whitegrid", font_scale=1.2)

plt.figure(figsize=(15, 6))

sns.barplot(
    data=summary_long,
    x="KL-Score",
    y="proportion_positive",
    hue="MRI_feature",
    palette="Set3"
    , width=0.6
)

plt.xlabel("KL-Score", fontsize=14)
plt.ylabel("Proportion Positive", fontsize=14)
plt.title("MRI Pathology Prevalence per KL-Score", fontsize=16)
plt.legend(
    title="MRI Finding",
    bbox_to_anchor=(1.05, 1),
    loc="upper left"
)
plt.tight_layout()
plt.show()

### Plots

In [ ]:
columns_corr =  ['mri_cart_yn', 'mri_osteo_yn', 'mri_bml_yn'] 
barplots(df, y_list=columns_corr, x='cluster_label', hue=None, figsize = (6, 6))

## Plot Distribution per cluster first

In [ ]:
feature_col = ['mri_bml_yn', 'mri_cart_yn', 'mri_osteo_yn', 'mri_syn_yn', 'mri_mnsc_yn', 'mri_lig_yn']

for feature in feature_col:
    clusters = df['cluster_label'].unique()
    clusters.sort()

    counts = df.groupby(['cluster_label', feature]).size().unstack(fill_value=0)

    plt.figure(figsize=(10, 6))
    counts.plot(kind='bar', stacked=False)
    plt.title(f'Distribution of {feature} across clusters')
    plt.xlabel('Cluster Label')
    plt.ylabel('Count')
    plt.show()